<a href="https://colab.research.google.com/github/sohv/ai_from_scratch/blob/main/genai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WGAN

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt

# Load MNIST dataset
(x_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
x_train = (x_train.astype(np.float32) - 127.5) / 127.5  # Normalize to [-1,1]
x_train = np.expand_dims(x_train, axis=-1)

# Model parameters
latent_dim = 100
img_shape = (28, 28, 1)
clip_value = 0.01
n_critic = 5

# Build Generator
def build_generator():
    model = Sequential([
        Dense(7 * 7 * 128, activation='relu', input_dim=latent_dim),
        Reshape((7, 7, 128)),
        BatchNormalization(),
        Conv2DTranspose(128, kernel_size=4, strides=2, padding='same', activation='relu'),
        BatchNormalization(),
        Conv2DTranspose(64, kernel_size=4, strides=2, padding='same', activation='relu'),
        BatchNormalization(),
        Conv2DTranspose(1, kernel_size=4, strides=1, padding='same', activation='tanh')
    ])
    return model

# Build Critic (Discriminator)
def build_critic():
    model = Sequential([
        Conv2D(64, kernel_size=4, strides=2, padding='same', input_shape=img_shape),
        LeakyReLU(alpha=0.2),
        Dropout(0.25),
        Conv2D(128, kernel_size=4, strides=2, padding='same'),
        LeakyReLU(alpha=0.2),
        Dropout(0.25),
        Flatten(),
        Dense(1)
    ])
    return model

# Create models
generator = build_generator()
critic = build_critic()
critic.compile(loss='mse', optimizer=Adam(0.0002, 0.5))

critic.trainable = False

gan_input = tf.keras.Input(shape=(latent_dim,))
gan_output = critic(generator(gan_input))
gan = Model(gan_input, gan_output)
gan.compile(loss='mse', optimizer=Adam(0.0002, 0.5))

# Training function
def train_wgan(epochs=10000, batch_size=128, sample_interval=1000):
    for epoch in range(epochs):
        for _ in range(n_critic):
            idx = np.random.randint(0, x_train.shape[0], batch_size)
            real_imgs = x_train[idx]
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            fake_imgs = generator.predict(noise)
            d_loss_real = critic.train_on_batch(real_imgs, -np.ones((batch_size, 1)))
            d_loss_fake = critic.train_on_batch(fake_imgs, np.ones((batch_size, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            for layer in critic.layers:
                for w in layer.trainable_weights:
                    w.assign(tf.clip_by_value(w, -clip_value, clip_value))
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        g_loss = gan.train_on_batch(noise, -np.ones((batch_size, 1)))
        if epoch % sample_interval == 0:
            print(f"Epoch {epoch}, D Loss: {d_loss}, G Loss: {g_loss}")
            sample_images(epoch)

# Function to generate images
def sample_images(epoch):
    noise = np.random.normal(0, 1, (10, latent_dim))
    generated_imgs = generator.predict(noise)
    generated_imgs = 0.5 * generated_imgs + 0.5  # Rescale to [0,1]
    plt.figure(figsize=(10, 2))
    for i in range(10):
        plt.subplot(1, 10, i+1)
        plt.imshow(generated_imgs[i, :, :, 0], cmap='gray')
        plt.axis('off')
    plt.show()

# Train WGAN
train_wgan(epochs=10000, batch_size=128, sample_interval=1000)

# DCGAN

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt

# Load MNIST dataset
(x_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
x_train = (x_train.astype(np.float32) - 127.5) / 127.5  # Normalize to [-1,1]
x_train = np.expand_dims(x_train, axis=-1)

# Model parameters
latent_dim = 100
img_shape = (28, 28, 1)

# Build Generator
def build_generator():
    model = Sequential([
        Dense(7 * 7 * 128, activation='relu', input_dim=latent_dim),
        Reshape((7, 7, 128)),
        BatchNormalization(),
        Conv2DTranspose(128, kernel_size=4, strides=2, padding='same', activation='relu'),
        BatchNormalization(),
        Conv2DTranspose(64, kernel_size=4, strides=2, padding='same', activation='relu'),
        BatchNormalization(),
        Conv2DTranspose(1, kernel_size=4, strides=1, padding='same', activation='tanh')
    ])
    return model

# Build Discriminator
def build_discriminator():
    model = Sequential([
        Conv2D(64, kernel_size=4, strides=2, padding='same', input_shape=img_shape),
        LeakyReLU(alpha=0.2),
        Conv2D(128, kernel_size=4, strides=2, padding='same'),
        LeakyReLU(alpha=0.2),
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    return model

# Create models
generator = build_generator()
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

discriminator.trainable = False

gan_input = tf.keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = tf.keras.Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

# Training function
def train_dcgan(epochs=10000, batch_size=128, sample_interval=1000):
    for epoch in range(epochs):
        idx = np.random.randint(0, x_train.shape[0], batch_size)
        real_imgs = x_train[idx]
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        fake_imgs = generator.predict(noise)
        d_loss_real = discriminator.train_on_batch(real_imgs, np.ones((batch_size, 1)))
        d_loss_fake = discriminator.train_on_batch(fake_imgs, np.zeros((batch_size, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))
        if epoch % sample_interval == 0:
            print(f"Epoch {epoch}, D Loss: {d_loss[0]}, G Loss: {g_loss}")
            sample_images(epoch)

# Function to generate images
def sample_images(epoch):
    noise = np.random.normal(0, 1, (10, latent_dim))
    generated_imgs = generator.predict(noise)
    generated_imgs = 0.5 * generated_imgs + 0.5  # Rescale to [0,1]
    plt.figure(figsize=(10, 2))
    for i in range(10):
        plt.subplot(1, 10, i+1)
        plt.imshow(generated_imgs[i, :, :, 0], cmap='gray')
        plt.axis('off')
    plt.show()

# Train DCGAN
train_dcgan(epochs=10000, batch_size=128, sample_interval=1000)

# GAN

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Reshape, LeakyReLU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt

# Load MNIST dataset
(x_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
x_train = (x_train.astype(np.float32) - 127.5) / 127.5  # Normalize to [-1,1]
x_train = np.expand_dims(x_train, axis=-1)

# Define model parameters
latent_dim = 100
img_shape = (28, 28, 1)

# Build Generator
def build_generator():
    model = Sequential([
        Dense(256, activation=LeakyReLU(alpha=0.2), input_dim=latent_dim),
        Dense(512, activation=LeakyReLU(alpha=0.2)),
        Dense(1024, activation=LeakyReLU(alpha=0.2)),
        Dense(28*28*1, activation='tanh'),
        Reshape(img_shape)
    ])
    return model

# Build Discriminator
def build_discriminator():
    model = Sequential([
        Flatten(input_shape=img_shape),
        Dense(512, activation=LeakyReLU(alpha=0.2)),
        Dense(256, activation=LeakyReLU(alpha=0.2)),
        Dense(1, activation='sigmoid')
    ])
    return model

# Compile models
generator = build_generator()
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

discriminator.trainable = False

gan_input = tf.keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

# Training function
def train_gan(epochs=10000, batch_size=128, sample_interval=1000):
    for epoch in range(epochs):
        # Train Discriminator
        idx = np.random.randint(0, x_train.shape[0], batch_size)
        real_imgs = x_train[idx]
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        fake_imgs = generator.predict(noise)
        d_loss_real = discriminator.train_on_batch(real_imgs, np.ones((batch_size, 1)))
        d_loss_fake = discriminator.train_on_batch(fake_imgs, np.zeros((batch_size, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train Generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

        # Print progress
        if epoch % sample_interval == 0:
            print(f"Epoch {epoch}, D Loss: {d_loss[0]}, G Loss: {g_loss}")
            sample_images(epoch)

# Function to generate and save images
def sample_images(epoch):
    noise = np.random.normal(0, 1, (10, latent_dim))
    generated_imgs = generator.predict(noise)
    generated_imgs = 0.5 * generated_imgs + 0.5  # Rescale to [0,1]

    plt.figure(figsize=(10, 2))
    for i in range(10):
        plt.subplot(1, 10, i+1)
        plt.imshow(generated_imgs[i, :, :, 0], cmap='gray')
        plt.axis('off')
    plt.show()

# Train GAN
train_gan(epochs=10000, batch_size=128, sample_interval=1000)

# AE

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt

# Load MNIST dataset
(x_train, _), (x_test, _) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

# Define encoding dimension
encoding_dim = 32

# Encoder
input_img = Input(shape=(28, 28, 1))
x = Flatten()(input_img)
x = Dense(128, activation='relu')(x)
x = Dense(encoding_dim, activation='relu')(x)
encoder = Model(input_img, x, name='encoder')

# Decoder
decoder_input = Input(shape=(encoding_dim,))
x = Dense(128, activation='relu')(decoder_input)
x = Dense(28*28, activation='sigmoid')(x)
outputs = Reshape((28, 28, 1))(x)
decoder = Model(decoder_input, outputs, name='decoder')

# Autoencoder model
autoencoder_output = decoder(encoder(input_img))
autoencoder = Model(input_img, autoencoder_output, name='autoencoder')

# Compile and train autoencoder
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(x_train, x_train, epochs=20, batch_size=128, validation_data=(x_test, x_test))

# Encode and decode test images
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

# Display original and reconstructed images
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28), cmap='gray')
    plt.axis('off')

    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28), cmap='gray')
    plt.axis('off')
plt.show()

# VAE

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda, Flatten, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.losses import mse
import numpy as np
import matplotlib.pyplot as plt

# Load MNIST dataset
(x_train, _), (x_test, _) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

# Define encoding dimension
latent_dim = 2

# Sampling function
def sampling(args):
    z_mean, z_log_var = args
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# Encoder
inputs = Input(shape=(28, 28, 1))
x = Flatten()(inputs)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')

# Decoder
decoder_inputs = Input(shape=(latent_dim,))
x = Dense(256, activation='relu')(decoder_inputs)
x = Dense(512, activation='relu')(x)
x = Dense(28 * 28, activation='sigmoid')(x)
outputs = Reshape((28, 28, 1))(x)

decoder = Model(decoder_inputs, outputs, name='decoder')

# VAE model
vae_outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, vae_outputs, name='vae')

# VAE loss function
reconstruction_loss = mse(Flatten()(inputs), Flatten()(vae_outputs)) * 28 * 28
kl_loss = -0.5 * tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=-1)
vae_loss = tf.reduce_mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

# Train the VAE
vae.fit(x_train, epochs=20, batch_size=128, validation_data=(x_test, None))

# Generate images from random samples
n = 10
grid_x = np.linspace(-2, 2, n)
grid_y = np.linspace(-2, 2, n)
plt.figure(figsize=(10, 10))
for i, yi in enumerate(grid_y):
    for j, xi in enumerate(grid_x):
        z_sample = np.array([[xi, yi]])
        x_decoded = decoder.predict(z_sample)
        digit = x_decoded[0].reshape(28, 28)
        ax = plt.subplot(n, n, i * n + j + 1)
        plt.imshow(digit, cmap='gray')
        plt.axis('off')
plt.show()